In [0]:
from pyspark.sql.types import (
    StructType, StructField, StringType, IntegerType, LongType, ShortType,
    ByteType, BooleanType, FloatType, DoubleType, DecimalType, DateType,
    TimestampType, BinaryType, ArrayType, MapType
)
import json
from datetime import datetime

class SchemaManager:
    def __init__(self, spark, metadata_table="ncp.metadata_table"):
        self.spark = spark
        self.metadata_table = f"{spark.catalog.currentCatalog()}.{metadata_table}"
        self.type_mapping = {
            "bigint": LongType(),
            "int": IntegerType(),
            "integer": IntegerType(),
            "tinyint": ByteType(),
            "long": LongType(),
            "smallint": ShortType(),
            "boolean": BooleanType(),
            "bit": BooleanType(),
            "decimal": lambda p, s: DecimalType(p, s),
            "numeric": lambda p, s: DecimalType(p, s),
            "float": FloatType(),
            "real": FloatType(),
            "double": DoubleType(),
            "char": StringType(),
            "string": StringType(),
            "varchar": StringType(),
            "nvarchar": StringType(),
            "text": StringType(),
            "date": DateType(),
            "datetime": TimestampType(),
            "timestamp": TimestampType(),
            "time": StringType(),
            "blob": BinaryType(),
            "binary": BinaryType()
        }
        self._create_metadata_table_if_not_exists()

    def _create_metadata_table_if_not_exists(self):
        """Ensure metadata table exists with required fields."""
        self.spark.sql(f"""
            CREATE TABLE IF NOT EXISTS {self.metadata_table} (
                table_name STRING,
                schema_json STRING,
                checkpoint TIMESTAMP,
                source_table STRING,
                table_keys STRING
            ) USING DELTA
        """)

    def update_metadata(self, table_name, field_name, field_value):
        """Generic function to update a metadata field."""
        valid_fields = {"schema_json", "checkpoint", "source_table", "table_keys"}
        if field_name not in valid_fields:
            raise ValueError(f"Invalid metadata field: {field_name}. Must be one of {valid_fields}.")

        # Format value based on type
        if isinstance(field_value, str):
            value_expr = f"'{field_value}'"
        elif isinstance(field_value, datetime):  # Convert datetime to timestamp
            value_expr = f"TIMESTAMP('{field_value.isoformat()}')"
        else:
            value_expr = str(field_value)

        self.spark.sql(f"""
            MERGE INTO {self.metadata_table} AS target
            USING (SELECT '{table_name}' AS table_name, {value_expr} AS {field_name}) AS source
            ON target.table_name = source.table_name
            WHEN MATCHED THEN UPDATE SET target.{field_name} = source.{field_name}
            WHEN NOT MATCHED THEN INSERT (table_name, {field_name}) 
            VALUES (source.table_name, source.{field_name})
        """)

    def add_schema(self, table_name, schema_dict):
        """Add or update a schema definition in the metadata table."""
        schema_json = json.dumps(schema_dict)
        self.update_metadata(table_name, "schema_json", schema_json)

    def get_schema(self, table_name):
        """Retrieve the schema as a StructType for a given table."""
        query = f"""
            SELECT schema_json FROM {self.metadata_table} 
            WHERE table_name = '{table_name}'
        """
        schema_row = self.spark.sql(query).collect()

        if not schema_row:
            print(f"Schema for table '{table_name}' not found.")
            return None

        schema_json = schema_row[0]["schema_json"]
        schema_dict = json.loads(schema_json)
        struct_fields = []
        for col_name, col_type in schema_dict.items():
            if "decimal" in col_type:
                p, s = map(int, col_type.replace("decimal(", "").replace(")", "").split(","))
                struct_fields.append(StructField(col_name, DecimalType(p, s), True))
            else:
                struct_fields.append(StructField(col_name, self.type_mapping[col_type], True))

        return StructType(struct_fields)

    def get_metadata(self, table_name, field_name):
        """Fetch any metadata field except schema_json."""
        valid_fields = {"checkpoint", "source_table", "table_keys"}
        if field_name not in valid_fields:
            raise ValueError(f"Invalid metadata field: {field_name}. Must be one of {valid_fields}.")

        query = f"""
            SELECT {field_name} FROM {self.metadata_table} 
            WHERE table_name = '{table_name}'
        """
        result = self.spark.sql(query).collect()

        if not result:
            print(f"No value found for '{field_name}' in table '{table_name}'.")
            return None

        return result[0][field_name]  # Directly return array, timestamp, or string

    def add_new_table_etl(self, schema_name, schema_dict, metadata_updates):

        # Add schema
        self.add_schema(schema_name, schema_dict)

        # Update metadata
        for key, value in metadata_updates.items():
            self.update_metadata(schema_name, key, value)

        # Get metadata values
        metadata = {key: schema_mgr.get_metadata(schema_name, key) for key in metadata_updates.keys()}
        metadata["schema"] = schema_mgr.get_schema(schema_name)
    
        return metadata

    def list_schemas(self):
        """List all table names that have schemas stored in the metadata table."""
        return [row["table_name"] for row in self.spark.sql(f"""
            SELECT table_name FROM {self.metadata_table}
        """).collect()]


In [0]:
import requests

class DatabricksSecretManager:
    def __init__(self):
        """
        Initializes the DatabricksSecretManager with a base URL and retrieves a personal access token.
        """
        base_url= dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().get()
        api_token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
        self.base_url = base_url.rstrip("/")
        self.token = api_token
        self.headers = {
            "Authorization": f"Bearer {self.token}",
            "Content-Type": "application/json"
        }

    def _post(self, endpoint, payload):
        url = f"{self.base_url}{endpoint}"
        response = requests.post(url, headers=self.headers, json=payload)
        return response

    def create_secret_scope(self, scope_name):
        """
        Creates a new secret scope.
        """
        payload = {"scope": scope_name}
        response = self._post("/api/2.0/secrets/scopes/create", payload)

        if response.status_code == 200:
            print(f"✅ Secret scope '{scope_name}' created successfully.")
        else:
            print(f"❌ Failed to create secret scope '{scope_name}'. Status: {response.status_code}, Response: {response.text}")

    def put_secret(self, scope, key, value):
        """
        Adds or updates a secret in the specified scope.
        """
        payload = {
            "scope": scope,
            "key": key,
            "string_value": value
        }
        response = self._post("/api/2.0/secrets/put", payload)

        if response.status_code == 200:
            print(f"✅ Secret '{key}' added to scope '{scope}' successfully.")
        else:
            print(f"❌ Failed to add secret '{key}'. Status: {response.status_code}, Response: {response.text}")

    def update_secret(self, scope, key, new_value):
        """
        Updates an existing secret key by overwriting it with a new value.
        """
        print(f"🔄 Updating secret '{key}' in scope '{scope}'...")
        self.put_secret(scope, key, new_value)
    
    def grant_permission(self, scope, principal, permission):
        """
        Grants permission on a secret scope to a user, group, or service principal.

        Args:
            scope (str): The secret scope name.
            principal (str): User or group to grant permissions (e.g. 'user@example.com' or 'users').
            permission (str): One of 'READ', 'WRITE', 'MANAGE'.
        """
        valid_permissions = {"READ", "WRITE", "MANAGE"}
        if permission not in valid_permissions:
            raise ValueError(f"Invalid permission '{permission}'. Must be one of {valid_permissions}")

        payload = {
            "scope": scope,
            "principal": principal,
            "permission": permission
        }
        response = self._post("/api/2.0/secrets/acls/put", payload)

        if response.status_code == 200:
            print(f"✅ Granted '{permission}' permission on scope '{scope}' to '{principal}'.")
        else:
            print(f"❌ Failed to grant permission. Status: {response.status_code}, Response: {response.text}")


In [0]:
def get_cloud_provider():
    browser_hostname = spark.conf.get("spark.databricks.workspaceUrl", "unknown")

    if "azuredatabricks.net" in browser_hostname:
        cloud = "Azure"
    elif "gcp" in browser_hostname:
        cloud = "GCP"
    elif "databricks" in browser_hostname or "amazonaws.com" in browser_hostname:
        cloud = "AWS"
    else:
        cloud = "Unknown"

    print(f"Running on: {cloud}")
    return cloud

In [0]:
# per = DatabricksSecretManager()

# per.create_secret_scope("azure-secret-scope")
# per.put_secret("azure-secret-scope", "AZURE_OPEN_AI_API_KEY_4_1", "")
# per.grant_permission("azure-secret-scope", "ARND/BA Users", "READ")
# per.grant_permission("azure-secret-scope", "Admins", "READ")